## Fashion Dataset Download

In [ ]:
# Google Drive
!gdown --id '1OqmmCIz1uwbj5RNkjpN_9Fnk6rdjJRES' --output fashion.zip

# Unzip the dataset.
# This may take some time.
!unzip -q fashion.zip

Downloading...
From: https://drive.google.com/uc?id=1OqmmCIz1uwbj5RNkjpN_9Fnk6rdjJRES
To: /content/fashion.zip
72.1MB [00:01, 67.6MB/s]


In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.models as model
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder
import struct

import pandas as pd
import torch.hub
from typing import Any

from tqdm.auto import tqdm

## 原始模型表現


In [ ]:
train_csv = pd.read_csv("fashion-mnist_train.csv")
test_csv = pd.read_csv("fashion-mnist_test.csv")

In [ ]:
train_csv.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,0,0,105,92,101,107,100,132,0,0,2,4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,150,...,211,220,214,74,0,255,222,128,0,0,0,0,0,0,0,0,0,44,12,0,0,40,134,162,191,214,163,146,165,79,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,0,114,183,112,55,23,72,102,165,160,28,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,24,188,163,93,...,171,249,207,197,202,45,0,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22,21,25,69,52,45,74,39,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,0,0,0,46,0,21,68,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25,187,189,...,230,237,229,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,116,112,136,147,144,121,102,63,0,0,0,0,0,0,0,0,0,0


In [ ]:
class FashionDataset(Dataset):
  def __init__(self, data, transform = None):
    """Method to initilaize variables.""" 
    self.fashion_MNIST = list(data.values)
    self.transform = transform
    
    label = []
    image = []
    
    for i in self.fashion_MNIST:
      label.append(i[0])
      image.append(i[1:])
    self.labels = np.asarray(label)
    print(self.labels)
    self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')

  def __getitem__(self, index):
    label = self.labels[index]
    image = self.images[index]
    
    if self.transform is not None:
      image = self.transform(image)

    return image, label

  def __len__(self):
    return len(self.images)

In [ ]:
batch_size = 100
train_set = FashionDataset(train_csv, transform=transforms.Compose([transforms.ToTensor()]))
test_set = FashionDataset(test_csv, transform=transforms.Compose([transforms.ToTensor()]))

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=8, pin_memory=True)
test_loader = DataLoader(train_set, batch_size=batch_size, num_workers=8, pin_memory=True)

[2 9 6 ... 8 8 7]
[0 1 2 ... 8 8 1]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
def output_label(label):
  output_mapping = {
    0: "T-shirt/Top",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat", 
    5: "Sandal", 
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot"
    }
  input = (label.item() if type(label) == torch.Tensor else label)
  return output_mapping[input]

In [ ]:
class AlexNet(nn.Module):
  def __init__(self, num_classes):
    super(AlexNet, self).__init__()
    self.features = nn.Sequential(
      nn.Conv2d(1, 32, kernel_size=3, padding=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=2, stride=2),
      nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
      nn.ReLU(inplace=True),
      nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
      nn.ReLU(inplace=True),
      nn.MaxPool2d(kernel_size=3, stride=2),
    )
    self.avgpool = nn.AdaptiveAvgPool2d((3, 3))
    self.classifier = nn.Sequential(
      nn.Dropout(),
      nn.Linear(256 * 3 * 3, 1024),
      nn.ReLU(inplace=True),
      nn.Dropout(),
      nn.Linear(1024, 512),
      nn.ReLU(inplace=True),
      nn.Linear(512, num_classes),
    )

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    x = self.features(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AlexNet(num_classes=10).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (x,y) in enumerate(dataloader):
    x, y = x.to(device), y.to(device)
    pred = model(x)
    loss = loss_fn(pred,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if batch%100==0:
    loss, current = loss.item(), batch * len(x)
    print(f"loss: {loss: >7f}[{curret: >5d}/{size: >5d}]")

def _test(dataloader, model):
  size = len(dataloader.dataset)
  model.eval()
  test_loss, correct = 0,0
  with torch.no_grad():
    for x, y in dataloader:
      x, y = x.to(device), y.to(device)
      pred = model(x)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()
  test_loss /= size
  correct /= size
  print(f"Test error: \n Acc: {correct}, Avg Loss:{test_loss}")

epoch = 10
for t in range(epoch):
  print(f"Epoch {t+1}\n-----------------")
  train(train_loader, model, loss_fn, optimizer)
  _test(test_loader, model)

Epoch 1
-----------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Test error: 
 Acc: 0.7327333333333333, Avg Loss:0.0074508494784434635
Epoch 2
-----------------
Test error: 
 Acc: 0.8069666666666667, Avg Loss:0.005352747803926468
Epoch 3
-----------------
Test error: 
 Acc: 0.83765, Avg Loss:0.004619357679784298
Epoch 4
-----------------
Test error: 
 Acc: 0.8541, Avg Loss:0.0041768316278855
Epoch 5
-----------------
Test error: 
 Acc: 0.8636666666666667, Avg Loss:0.003865952519575755
Epoch 6
-----------------
Test error: 
 Acc: 0.8709666666666667, Avg Loss:0.0036320103374620277
Epoch 7
-----------------
Test error: 
 Acc: 0.87735, Avg Loss:0.0034474132421116036
Epoch 8
-----------------
Test error: 
 Acc: 0.8822666666666666, Avg Loss:0.0033034680505593618
Epoch 9
-----------------
Test error: 
 Acc: 0.88635, Avg Loss:0.0031773423805832863
Epoch 10
-----------------
Test error: 
 Acc: 0.8904166666666666, Avg Loss:0.0030648924012978873


## 取得目標malware執行檔轉浮點數

In [ ]:
import struct
with open('/content/Virus.vbs',mode = 'r') as f:
    file = f.read()

def str_to_byte(x):
    num = ord(x)
    byte = chr(num)
    return byte

bytes_ = []
for i in file:
    byte = str_to_byte(i)
    bytes_ = bytes_ + [byte]

bytes_list = []  
prefix = ['\x3c']
for i in range(0,270):
    comb = prefix+bytes_[i*3:(i+1)*3]
    bytes_list = bytes_list + [comb]

new_list = []
for i in bytes_list:
    word = ''
    for j in i:
        word = word + j
    bword = bytes(word, 'utf-8')
    param = struct.unpack(">f", bword)
    new_list = new_list + [param[0]]
malFloat = new_list

In [ ]:
malFloat

### 尋找可替換的神經元

In [ ]:
model_ft = model
model_ft.parameters()
model_dict = model_ft.state_dict()
model_dict.keys()

odict_keys(['features.0.weight', 'features.0.bias', 'features.3.weight', 'features.3.bias', 'features.6.weight', 'features.6.bias', 'features.8.weight', 'features.8.bias', 'features.10.weight', 'features.10.bias', 'classifier.1.weight', 'classifier.1.bias', 'classifier.4.weight', 'classifier.4.bias', 'classifier.6.weight', 'classifier.6.bias'])

In [ ]:
fullyConn = []
for i in model_dict.keys():
    if 'classifier' in i and 'weight' in i:
        fullyConn.append(i)
fullyConn

['classifier.1.weight', 'classifier.4.weight', 'classifier.6.weight']

In [ ]:
decode_conv = []
decode_j = []
decode_k = []
codes = []
for param in malFloat:
  print(param)
  b = ''
  for i in fullyConn:
    if b == 'break':
      break
    for j in range(len(model_dict[i])):
      if b == 'break':
        break
      for k in range(len(model_dict[i][j])):
        neuron = model_dict[i][j][k]
        code = i+str(j)+str(k)
        if code not in codes:
          if abs(neuron.cpu().numpy().tolist()-param)<=0.0001:
            decode_conv = decode_conv + [i]
            decode_j = decode_j + [j]
            decode_k = decode_k + [k]
            codes = codes + [code]
            b = 'break'
        if b == 'break':
          break

0.01319820899516344
0.014856161549687386
0.014616049826145172
0.009909017942845821
0.014980822801589966
0.013881546445190907
0.014676440507173538
0.014004260301589966
0.014248639345169067
0.014247512444853783
0.01400076225399971
0.014000923372805119
0.009834559634327888
0.010509531013667583
0.010816138237714767
0.009907820262014866
0.014918421395123005
0.00989747978746891
0.01319820899516344
0.014856161549687386
0.014616049826145172
0.009909017942845821
0.014980822801589966
0.013881546445190907
0.014676440507173538
0.014004260301589966
0.014248639345169067
0.014247512444853783
0.01400076225399971
0.014000923372805119
0.009834559634327888
0.010509531013667583
0.010816138237714767
0.009907820262014866
0.014918421395123005
0.00989747978746891
0.01319820899516344
0.014856161549687386
0.014616049826145172
0.009909017942845821
0.014980822801589966
0.013881546445190907
0.014676440507173538
0.014004260301589966
0.014248639345169067
0.014247512444853783
0.01400076225399971
0.014000923372805119


In [ ]:
# 將可替換神經元座標存成.csv檔案
Infected_Neurons = pd.DataFrame({
    'layer':decode_conv,
    'param1':decode_j,
    'param2':decode_k,
})
Infected_Neurons.to_csv('Infected_Neurons.csv',index = None)

## 開始替換神經元

In [ ]:
new_model = model_ft.state_dict().copy()
for i in range(len(Infected_Neurons)):
  layer = Infected_Neurons['layer'][i]
  param1 = Infected_Neurons['param1'][i]
  param2 = Infected_Neurons['param2'][i]
  new_param = torch.Tensor([malFloat[i]])[0]
  new_model[layer][param1][param2] = new_param
torch.save(new_model,'Infected_Model.pt') # 受損的模型權重檔案

### 測試替換後模型表現

In [ ]:
model_ft.load_state_dict(new_model)

<All keys matched successfully>

In [ ]:
_test(test_loader, model_ft)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test error: 
 Acc: 0.8904166666666666, Avg Loss:0.0030649043987194696


## 結論
### 我們成功替換了所有受損神經元，且模型預測能力完全沒有下降。